In [48]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    groq_api_key="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",
    model="llama-3.1-70b-versatile"
)
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-39383")
page_data = loader.load().pop().page_content
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys:`company name`,`role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
import pandas as pd

df = pd.read_csv("roles.csv")

import uuid
for _, row in df.iterrows():
    collection.add(documents=row["Techstack"],
        metadatas={"links": row["Links"]}, ids=[str(uuid.uuid4())])

# Use extracted skills from json_res
job = json_res

# Check if 'skills' key exists in the job and is not empty
if 'skills' in job and job['skills']:
    links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
else:
    links = []  # Handle case where no skills are available

prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Divya Dadi, a Master of Computer Science student at Rice University with internship experience as a Software Engineer at Walmart Global Tech and a Junior Data Analyst at Tata Consultancy Services. 
        At Walmart Global Tech, I processed large datasets using Scala and Apache Spark, ensuring high performance and scalability. I also designed and implemented automated data pipelines with Apache Airflow, ensuring timely availability of critical data. 
        Additionally, I leveraged GCP services, such as BigQuery and Google Cloud Storage, for efficient data storage and retrieval.I collaborated closely with cross-functional teams to maintain data accuracy and integrity. Furthermore, I developed comprehensive Tableau dashboards, providing valuable insights and supporting data-driven decision-making across the organization.
        Your job is to write a cold email to the hiring manager regarding the above-mentioned job, describing your skills and experiences that align with the job requirements. Don't involve any other links.
        Remember you are Divya Dadi, a student and a candidate for the position. Please include link to my LinkedIn: https://www.linkedin.com/in/divya-dadi-9a2539172/
        ### EMAIL (NO PREAMBLE):
        
        """
)

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Software Engineer II - Workday Integration, ITC at NIKE, INC.

Dear Hiring Manager,

I am excited to apply for the Software Engineer II - Workday Integration, ITC role at NIKE, INC. As a Master of Computer Science student at Rice University with a strong background in software engineering and data analysis, I am confident that my skills and experiences align with the job requirements.

With over 2 years of experience in software engineering and data analysis, I have developed a strong foundation in object-oriented concepts, which I believe is essential for this role. My experience with Scala and Apache Spark has taught me the importance of high performance and scalability in software development. Additionally, my experience with Apache Airflow has given me a deep understanding of automated data pipelines and timely data availability.

Although I don't have direct experience with Workday Studio, EIB, Core Connectors, and payroll connectors (PECI), I am eager to 

In [56]:
# Import necessary libraries and modules
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import pandas as pd
import uuid

# Step 1: Initialize the LLM (ChatGroq) with specific model and API key
llm = ChatGroq(
    temperature=0,  # Set temperature for response randomness
    groq_api_key="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",  # API key for authentication
    model="llama-3.1-70b-versatile"  # The specific LLM model to use
)

# Step 2: Load the webpage data from the specified URL (Nike job page)
loader = WebBaseLoader("https://jobs.nike.com/job/R-39383")
page_data = loader.load().pop().page_content  # Get the content from the loaded page

# Step 3: Define the prompt template to extract job postings in JSON format
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the 
    following keys:`company name`,`role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):    
    """
)

# Step 4: Run the extraction chain to get job postings in JSON format
chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data': page_data})

# Step 5: Parse the extracted response into valid JSON
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)  # Convert the LLM output into a JSON object

# Step 6: Load roles.csv for tech stack data (techstack and links)
df = pd.read_csv("roles.csv")  # Read CSV file into a pandas DataFrame

# Step 7: (Optional) Add data from the CSV to ChromaDB collection (for advanced querying)
client = chromadb.PersistentClient('vectorstore')  # Initialize ChromaDB client
collection = client.get_or_create_collection(name="portfolio")  # Create or get existing collection

# Add rows from CSV into the ChromaDB collection
for _, row in df.iterrows():
    collection.add(documents=row["Techstack"],
                   metadatas={"links": row["Links"]}, 
                   ids=[str(uuid.uuid4())])  # Add rows to the collection

# Step 8: Use the extracted skills from the JSON result to query ChromaDB
job = json_res  # Assign parsed job details from the JSON result

# Check if 'skills' key exists in the job and fetch relevant links from ChromaDB
if 'skills' in job and job['skills']:
    links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
else:
    links = []  # Handle the case where no skills are available or 'skills' key is missing

# Step 9: Define the email prompt template using job description and links
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Divya Dadi, a Master of Computer Science student at Rice University with internship experience as a Software Engineer at Walmart Global Tech and a Junior Data Engineer at Tata Consultancy Services. 
    At Walmart Global Tech, I processed large datasets using Scala and Apache Spark, ensuring high performance and scalability. I also designed and implemented automated data pipelines with Apache Airflow, ensuring timely availability of critical data. 
    Additionally, I leveraged GCP services, such as BigQuery and Google Cloud Storage, for efficient data storage and retrieval.I collaborated closely with cross-functional teams to maintain data accuracy and integrity. Furthermore, I developed comprehensive Tableau dashboards, providing valuable insights and supporting data-driven decision-making across the organization.
    
    Your job is to write a cover letter to the hiring manager regarding the above-mentioned job, describing your skills and experiences that align with the job requirements. Don't involve any other links.
    Remember you are Divya Dadi, a student and a candidate for the position. Please include link to my LinkedIn: https://www.linkedin.com/in/divya-dadi-9a2539172/. 
    
    ### EMAIL (NO PREAMBLE):
    
    """
)

# Step 10: Generate the email using the LLM and the job description
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})

# Step 11: Output the generated email content
print(res.content)


Subject: Application for Software Engineer II - Workday Integration, ITC at NIKE, INC.

Dear Hiring Manager,

I am excited to apply for the Software Engineer II - Workday Integration, ITC position at NIKE, INC. As a Master of Computer Science student at Rice University with a strong background in software engineering and data engineering, I am confident that my skills and experiences align with the job requirements.

With over 2 years of experience in software engineering and data engineering, I have developed a strong foundation in object-oriented concepts, which I believe is essential for this role. My experience as a Software Engineer at Walmart Global Tech has provided me with hands-on experience in designing and implementing automated data pipelines using Apache Airflow, ensuring timely availability of critical data. Additionally, I have leveraged GCP services, such as BigQuery and Google Cloud Storage, for efficient data storage and retrieval.

Although I don't have direct experi